In [ ]:
%pip install faiss-cpu beautifulsoup4

In [1]:
import os
from dotenv import load_dotenv


load_dotenv()

True

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
# Load docs
# 지정한 URL의 웹페이지 내용을 가져와서 Document 객체 리스트로 반환
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load() # data는 Document 객체 리스트

# Split
# 한 청크당 최대 500자, 청크 간 중복은 0자로 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# 전체 문서를 청크 단위로 분할
# split_documents는 각 Document를 최대 500자씩 나누어 Document 리스트로 반환
all_splits = text_splitter.split_documents(data)

# Stroe splits
# OpenAIEmbeddings를 사용해 각 청크를 임베딩하고 FAISS 인덱스에 저장
# 생성된 vectorstore는 검색 시 유사도 검색 기능을 제공
vectorstore = FAISS.from_documents(
    documents=all_splits, # 분할된 문서 청크 리스트
    embedding=OpenAIEmbeddings() # 임베딩 함수
    )

# LLM
# ChatOpenAI 인스턴스를 생성하여 LLM(언어 모델) 호출 준비
llm = ChatOpenAI()

In [4]:
## Legacy

from langchain import hub
from langchain.chains import RetrievalQA


LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')

prompt = hub.pull('rlm/rag-prompt')

qa_chain = RetrievalQA.from_llm(
    llm, # -> LLM 모델 설정
    retriever=vectorstore.as_retriever(), # -> 유사도 검색 
    prompt=prompt, # -> 사용자 질문
)

qa_chain('what are autonmous agents?')

C:\Users\2\AppData\Local\Temp\ipykernel_6040\1393769394.py:17: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa_chain('what are autonmous agents?')


{'query': 'what are autonmous agents?',
 'result': 'Autonomous agents are systems empowered by large language models (LLM) to perform tasks like scientific discovery, experimental design, and problem-solving independently. These agents can browse the internet, read documentation, execute code, and utilize other LLMs for various tasks. LLM serves as the central controller or brain of autonomous agents, enabling them to function as powerful problem solvers.'}

In [8]:
## LCEL

from langchain import hub
# LangChain 허브에서 사전 제작된 프롬프트(preset prompt) 또는 리소스를 가져오기 위한 모듈
from langchain_core.output_parsers import StrOutputParser
# 결과를 문자열로 파싱하기 위한 파서
from langchain_core.runnables import RunnablePassthrough
# 입력을 그대로 다음 단계로 전달할 때 사용하는 러너블

# 1. 허브에서 RAG(검색-증강 생성)용 프롬프트 설정을 불러옵니다.
prompt = hub.pull('rlm/rag-prompt')
#    - 'rlm/rag-prompt'는 LangChain 허브에 저장된 프롬프트 템플릿 식별자입니다.
#    - 이를 통해 문서 컨텍스트와 질문을 조합하는 방식 등, 모범적으로 설계된 RAG 프롬프트를 가져옵니다.

# 2. 문서 목록을 받아서 하나의 문자열 블록으로 합치는 헬퍼 함수 정의
def format_docs(docs):
    """
    docs: Document 객체들의 리스트
    반환값: 각 문서의 page_content를 
           두 줄 바꿈('\n\n')으로 구분하여 하나의 긴 문자열로 결합
    """
    return '\n\n'.join(doc.page_content for doc in docs)

# 3. QA 체인 구성
qa_chain = (
    {
        # 'context' 필드에는 VectorStore에서 검색된 문서 청크를,
        # format_docs 함수에 파이프 연산자로 전달하여 문자열로 변환
        'context': vectorstore.as_retriever() | format_docs,
        # 'question' 필드에는 질문을 그대로 다음 단계로 전달하는 러너블 사용
        'question': RunnablePassthrough(),
    }
    | prompt # 4. 프롬프트 템플릿에 context와 question을 넣어 완전한 프롬프트 생성
    | llm    # 5. 생성된 프롬프트를 LLM(언어 모델)에 전달하여 응답 생성
    | StrOutputParser() # 6. LLM의 출력을 문자열로 파싱
)

# 7. 체인 실행: 사용자 질문("What are autonomous agents?")을 invoke하여 최종 답변을 얻음
qa_chain.invoke('What are autonomous agents?')

'Autonomous agents are agents that can handle autonomous design, planning, and performance of complex tasks without direct human intervention. These agents can use tools to browse the Internet, read documentation, execute code, and leverage other language models for decision-making. They can call external APIs for additional information needed for their tasks.'